In [1]:
import random
import pandas as pd
from datetime import datetime

# Temporary workflow
# (1) Download csv with `heroku run make exportdb && make importdb`, load it here
# (2) save links to ~/Downloads/links.txt in onetab format and run this script (or otherwise modify the CSV as desired)
# (3) Upload back up with `make csv_to_s3`
# (4) Run `heroku run make importdb` to get it to production
links = pd.read_csv('data/export.csv')
links

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""""""""""""""""""""""""""""""&nbsp;&ldquo;People disagree ab...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1,ethics,custom,17,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""""""""""""""""""""""""""""""&nbsp;&ldquo;Why give globally?...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1,giving,custom,59,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""""""""""""""""""""""""""""""&nbsp;&ldquo;I&rsquo;m fascinat...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1,productivity,custom,56,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""""""""""""""""""""""""""""""&nbsp;&ldquo;In Glen&rsquo;s vo...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1,politicalscience,custom,29,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here&rsqu...,"""""""""""""""""""""""""""""""&nbsp;&ldquo;August 13\, 6:40 P...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1,technology,custom,92,0
...,...,...,...,...,...,...,...,...,...,...,...,...
76014,132474,https://twitter.com/peterwildeford/status/1433...,"Whenever I get a service window\, like ""our te...",\N,twitter.com,2021-09-07 03:48:15.807211+00,2021-09-07 03:48:15.807197+00,1,forecasting,Custom,1,1
76015,132475,https://twitter.com/avi_eisen/status/143383764...,7 habits of highly successful people: 1. Lose...,\N,twitter.com,2021-09-07 03:48:35.02648+00,2021-09-07 03:48:35.026453+00,1,productivity,Custom,1,1
76016,132476,https://twitter.com/Jotto999/status/1433827323...,Something I was wrong about too. Suicide [dur...,\N,twitter.com,2021-09-07 03:49:02.331402+00,2021-09-07 03:49:02.331396+00,1,policy,Custom,1,1
76017,132477,https://twitter.com/kwcollins/status/143382222...,Hypothesis: The American public generally only...,\N,twitter.com,2021-09-07 03:49:20.120429+00,2021-09-07 03:49:20.120421+00,1,politicalscience,Custom,1,1


In [2]:
links['added'].max()

'2021-09-07 11:07:49.993033+00'

In [3]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]['id']

73076    128763
73077    128764
73078    128765
73079    128766
73080    128767
          ...  
75448    131442
75449    131444
75450    131445
75451    131446
75452    131447
Name: id, Length: 2375, dtype: int64

In [4]:
link_file = open('/Users/peterhurford/Downloads/links.txt', 'r')
new_links = link_file.readlines()

In [5]:
def new_entry(links, url, title):
    return {'id': links['id'].max() + 1,
            'url': str(url).strip().replace(',', ''),
            'title': str(title).strip().replace(',', ''),
            'summary': '\\N',
            'domain': '\\N',
            'added': str(datetime.now()),
            'modified': str(datetime.now()),
            'liked': '\\N',
            'category': '\\N',
            'aggregator': 'Custom',
            'seed': random.randint(1, 100),
            'tweet': 0}

for i, new_link in enumerate(new_links):
    if i % 100 == 0:
        print('...{}/{}'.format(i + 1, len(new_links)))
    if ' | ' in new_link:
        new_link = new_link.replace('    - ', '').replace('\n', '')
        entry = new_entry(links, new_link.split(' | ')[0], new_link.split(' | ')[1])
        links = links.append(entry, ignore_index=True)

links

...1/5


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""""""""""""""""""""""""""""""&nbsp;&ldquo;People disagree ab...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1,ethics,custom,17,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""""""""""""""""""""""""""""""&nbsp;&ldquo;Why give globally?...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1,giving,custom,59,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""""""""""""""""""""""""""""""&nbsp;&ldquo;I&rsquo;m fascinat...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1,productivity,custom,56,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""""""""""""""""""""""""""""""&nbsp;&ldquo;In Glen&rsquo;s vo...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1,politicalscience,custom,29,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here&rsqu...,"""""""""""""""""""""""""""""""&nbsp;&ldquo;August 13\, 6:40 P...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1,technology,custom,92,0
...,...,...,...,...,...,...,...,...,...,...,...,...
76019,132479,https://www.nytimes.com/2021/07/24/opinion/cor...,Opinion,\N,\N,2021-09-07 06:26:50.546357,2021-09-07 06:26:50.546388,\N,\N,Custom,16,0
76020,132480,https://thebrowser.com/notes/applied-divinity-...,Browser Interviews: Applied Divinity Studies,\N,\N,2021-09-07 06:26:50.560990,2021-09-07 06:26:50.561001,\N,\N,Custom,8,0
76021,132481,https://forum.effectivealtruism.org/posts/z9hz...,Takeaways on US Policy Careers (Part 1): Paths...,\N,\N,2021-09-07 06:26:50.576342,2021-09-07 06:26:50.576354,\N,\N,Custom,8,0
76022,132482,https://ettoresoftware.store/mac-apps/typeit4me/,TypeIt4Me: Try the original text expander for ...,\N,\N,2021-09-07 06:26:50.590578,2021-09-07 06:26:50.590591,\N,\N,Custom,60,0


In [6]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet
73076,128763,https://www.cs.utexas.edu/users/dahlin/bookshe...,Dr. R. W. Hamming's Advice on Research,\N,\N,2021-08-03 10:19:48.01014+00,2021-08-03 10:19:48.010157+00,\N,\N,Custom,86,0
73077,128764,https://autotranslucence.wordpress.com/2018/03...,Becoming a magician – Autotranslucence,\N,\N,2021-08-03 10:19:48.022198+00,2021-08-03 10:19:48.022222+00,\N,\N,Custom,93,0
73078,128765,https://www.sup.org/books/title/?id=26137,The Case for U.S. Nuclear Weapons in the 21st ...,\N,\N,2021-08-03 10:19:48.034344+00,2021-08-03 10:19:48.034396+00,\N,\N,Custom,17,0
73079,128766,https://www.belfercenter.org/sites/default/fil...,untitled,\N,\N,2021-08-03 10:19:48.049316+00,2021-08-03 10:19:48.049333+00,\N,\N,Custom,45,0
73080,128767,https://forecastapp.substack.com/p/whos-best-a...,Who's Best at Predicting the Future? - by Jare...,\N,\N,2021-08-03 10:19:48.062958+00,2021-08-03 10:19:48.062972+00,\N,\N,Custom,84,0
...,...,...,...,...,...,...,...,...,...,...,...,...
76019,132479,https://www.nytimes.com/2021/07/24/opinion/cor...,Opinion,\N,\N,2021-09-07 06:26:50.546357,2021-09-07 06:26:50.546388,\N,\N,Custom,16,0
76020,132480,https://thebrowser.com/notes/applied-divinity-...,Browser Interviews: Applied Divinity Studies,\N,\N,2021-09-07 06:26:50.560990,2021-09-07 06:26:50.561001,\N,\N,Custom,8,0
76021,132481,https://forum.effectivealtruism.org/posts/z9hz...,Takeaways on US Policy Careers (Part 1): Paths...,\N,\N,2021-09-07 06:26:50.576342,2021-09-07 06:26:50.576354,\N,\N,Custom,8,0
76022,132482,https://ettoresoftware.store/mac-apps/typeit4me/,TypeIt4Me: Try the original text expander for ...,\N,\N,2021-09-07 06:26:50.590578,2021-09-07 06:26:50.590591,\N,\N,Custom,60,0


In [7]:
links['id'] = links['id'].astype(int)
links['seed'] = links['seed'].astype(int)
links['tweet'] = links['tweet'].apply(lambda x: 0 if str(x) == '\\N' else str(x).split('.')[0]).astype(int)

In [8]:
links.to_csv('data/export.csv', index=False)